In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import os
import scipy.sparse as sparse
from test import *

from itertools import chain
from pathlib import *
from math import ceil
from sqlalchemy import create_engine
from pandarallel import pandarallel

from test import *
from helper import *
from tree_weight import *
from time import time
from main import make_graph_from_tree
from multiprocessing import *

from typing import List
%matplotlib inline

In [2]:
pandarallel.initialize(nb_workers=cpu_count() - 1)

INFO: Pandarallel will run on 3 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
image_folder = 'graphs/'
tree = 'benchmark_models/nonogram/trees/dom_06.sqlite'
info_df = to_df(tree, 'info').set_index('NodeID')
nodes_df = to_df(tree, 'nodes').set_index('NodeID')
valid_df = nodes_df[nodes_df['Status'] != 3]

In [4]:
state_dict = {
    'nodes_df': nodes_df,
    'info_df': info_df,
    'tree': tree,
    'weight_scheme': 'searchSpace_scheme',
    'weight_colname': 'NodeWeight',
    'assign_in_dfs_order': False,
    'use_parallel': True
}

assign_weight(state_dict)

In [9]:
(nodes_df.groupby(['ParentID'])['NodeWeight'].nunique() != 1).sum()

0

In [44]:
j.loc[j.index] = j
j

ParentID
0       158
1       158
2       158
4         2
5         2
       ... 
4730      2
4731      2
4736      2
4738      2
4740      2
Name: NodeID, Length: 2371, dtype: int64

In [122]:
k = 0.9
res_df = pd.DataFrame.copy(valid_df.iloc[1:, :])
res_df['Weight'] = np.random.random(res_df.shape[0])
parent_mean = res_df.groupby('ParentID').mean()['Weight']
parent_count = res_df.groupby('ParentID').count()['Weight']

# index swap
res_df = res_df.reset_index().set_index('ParentID')
res_df.loc[:, 'Mean'] = parent_mean
res_df.loc[:, 'Count'] = parent_count
res_df = res_df.reset_index().set_index('NodeID')
res_df.loc[:, 'Weight'] = res_df['Weight'] - res_df['Mean'] + k / res_df['Count']
j = res_df.groupby(['ParentID']).sum()['Weight']
j[abs(j - k) > 1e-10]

Series([], Name: Weight, dtype: float64)

In [119]:
weights = valid_df[['ParentID', 'HasUnequalSplit']].reset_index().set_index('ParentID').iloc[1:,:]
weights.loc[:, 'ParentDomainSize'] = parent_domain_size
weights = weights.reset_index().set_index('NodeID')
weights['Weight'] = 1 / weights['ParentDomainSize'] + weights['HasUnequalSplit'] * (1 - 2 / weights['ParentDomainSize'])
weights.drop(columns)

,ParentID,HasUnequalSplit,ParentDomainSize,Weight
NodeID,,,,
1,0,False,694,0.001441
2,0,True,694,0.998559
3,1,False,694,0.001441
4,1,True,694,0.998559
5,3,False,684,0.001462
...,...,...,...,...
31278,31274,True,1,0.000000
31279,31278,False,1,1.000000
31280,31278,True,1,0.000000


In [48]:
weights

array([0.00144092, 0.99855908, 0.00144092, ..., 0.        , 1.        ,
       0.        ])

In [10]:
domains

,Info,label
NodeID,,
0,"{'objective': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1...",NaN
1,"{'objective': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1...",X_INTRODUCED_0_
2,"{'objective': {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ...",X_INTRODUCED_0_
3,"{'objective': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1...",X_INTRODUCED_2_
4,"{'objective': {10, 11, 12, 13, 14, 15, 16, 17,...",X_INTRODUCED_2_
...,...,...
31278,"{'objective': {693}, 'fuel': {1542, 1543, 1544...",X_INTRODUCED_3_
31279,"{'objective': {693}, 'fuel': {1984, 1985, 1986...",X_INTRODUCED_3_
31280,"{'objective': {693}, 'fuel': {1938, 1939, 1940...",X_INTRODUCED_3_


In [14]:
nodeSplitVarDomain = domains.iloc[1:,:].parallel_apply(lambda r: len(r['Info'][r['label']]), axis=1)

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
domains['parentLabel'] = get_parent_column('label', domains)

In [ ]:
domains

In [ ]:
print(info_df.shape)